In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Panel
import csv
import matplotlib.pyplot as plt
from collections import Counter
import sklearn
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import log_loss
import datetime
from random import random
from collections import defaultdict
from numpy import isnan
import numpy
import operator
from scipy import spatial
from operator import truediv
from pylab import *

In [861]:
import random as rand
rand.randrange(1,5,1)

3

### Glogal features

In [3]:
impression = pd.read_pickle('../data/processed/joined_impressions.pkl').fillna(0)

In [4]:
impression['date'] = impression['impressionTimestamp'].dt.date

In [5]:
articles = pd.read_pickle('../data/processed/articles.pkl')

In [6]:
satisfiedlist = []
for i in list(impression.maxDuration):
    if i >= 10000:
        satisfiedlist.append(1)
    else:
        satisfiedlist.append(0)

impression['satisfiedlist'] = satisfiedlist        
    

In [7]:
category_flatten = [val for sublist in articles.categories.values for val in sublist]

category_flatten = list(set(category_flatten))

In [8]:
rich_impression = impression.merge(articles, left_on="contentId",right_on="id")

In [9]:
from sklearn import preprocessing
from itertools import chain

def columns_to_key_feature_pairs(row, key_column, feature_columns):
    return [(row[key_column], '{}={}'.format(column, row[column])) for column in feature_columns]

def array_column_to_key_feature_pairs(row, key_column, array_column):
    return [(row[key_column], u'{}={}'.format(array_column, value)) for value in row[array_column]]

feature_columns = ['id', 'hotness', 'lifetime']

item_features = pd.DataFrame.from_records(
    data=chain.from_iterable(
        columns_to_key_feature_pairs(row, key_column='id', feature_columns=feature_columns) +\
        array_column_to_key_feature_pairs(row, key_column='id', array_column='categories') +\
        array_column_to_key_feature_pairs(row, key_column='id', array_column='tags')
        for _, row in articles.iterrows()), 
    columns=['item', 'feature_name'],
    index='item')

#item_features.head()

In [10]:
max_position = impression.contentPosition.max()
content_positions = np.arange(max_position+1)

context_features = pd.DataFrame.from_dict(
    {'contentPosition': content_positions, 'feature_name': ['position={}'.format(p) for p in content_positions]}
).set_index('contentPosition')

#context_features

In [11]:
all_features = pd.concat([item_features, context_features],axis=0)

all_features.feature_name = all_features.feature_name.astype('category')
all_features['encoded_feature'] = all_features.feature_name.cat.codes

#all_features.head()

### user with category

In [12]:
environmentlist = list(set(rich_impression["environmentId"].values))

In [13]:
grouped = rich_impression.groupby(["environmentId"])
dic = {}
for item in environmentlist:
    l = grouped.get_group(item)["categories"].tolist()
    flatten =  list(set([item for sublist in l for item in sublist]))
    dic[item] = flatten
    
    

In [14]:
userinfor = pd.DataFrame({"environmentid": list(dic.keys()),"category": list(dic.values())})

flat_userinfor = pd.DataFrame(
    data=[(index, category)
         for index, row in userinfor.iterrows()
         for category in row.category],
    columns=['index', 'category']).set_index("index")

### we cluster different users into 20 groups based on interests

In [15]:
from sklearn.preprocessing import LabelEncoder 

from scipy import sparse

import scipy.sparse as sp

def feature_matrix(rich_impressions, column_name):
    
    category_encoder = LabelEncoder()

    rich_impressions = rich_impressions.assign(encoded_column=lambda df: category_encoder.fit_transform(df[column_name]))

    row_indexes = rich_impressions.index.values    
    column_indexes = rich_impressions.encoded_column.values
    
    output = sparse.coo_matrix(
        (np.ones_like(column_indexes), 
        (row_indexes, column_indexes)))    
    
    return output
    

In [16]:
from sklearn.preprocessing import OneHotEncoder

def singletag(mergedate, name):
    
    enc = OneHotEncoder()

    X = enc.fit_transform(mergedate[[name]].values)
    
    #result = X.toarray()
    
    return X
    

In [17]:

from sklearn.cluster import KMeans  

def kmeans(X, n_clusters):

    labeler = KMeans(n_clusters) 

    labeler.fit(X.tocsr())  
    
    return labeler


In [18]:
def indices( mylist, value):
    return [i for i,x in enumerate(mylist) if x==value]


In [19]:
X_user = feature_matrix(flat_userinfor, "category")

labeler = kmeans(X_user,20)


### Select just one groups to do the experient

In [746]:
index_6 = indices(labeler.labels_, 11)

### Find rich_impression  data set for each group

In [747]:
def grouped_rich_impression(indexlist, userinfor, rich_impression):
    idx = userinfor.index.isin(indexlist)
    groupeduserinf = userinfor[idx]
    groupedenvironment = groupeduserinf.environmentid.values.tolist()
    im_idx = rich_impression.environmentId.isin(groupedenvironment)
    group_rich_impression = rich_impression[im_idx]
    return groupeduserinf,group_rich_impression
    

In [748]:
groupeduserinf,grouped_rich_impression = grouped_rich_impression(index_6, userinfor, rich_impression)



### There are seven days, we can seen as seven frontpages

In [749]:
def CTRforday(grouped_rich_impression):

    up = grouped_rich_impression.groupby(["date"]).aggregate({"isClicked":sum})
    
    down = grouped_rich_impression.groupby(["date"]).aggregate({"isClicked":len})
    
    CTR = up.isClicked.values/down.isClicked.values
    
    up["CTR"] = CTR
    
    result = up.sort(columns="CTR")
    
    return result

In [750]:
result = CTRforday(grouped_rich_impression) 

result

/Users/XiHUANG/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,isClicked,CTR
date,,
2017-06-18,399.0,0.020440
2017-06-19,162.0,0.023028
2017-06-14,99.0,0.030165
2017-06-15,66.0,0.033183
2017-06-16,64.0,0.035301
2017-06-13,86.0,0.036013
2017-06-17,104.0,0.044199


### seperate the dataset, train is  based on (2017-06-13 ~ 2017-06-18), test is based on (2017-06-19)

In [751]:
def selecttraintest(impression,n):
        impression["index"] = pd.to_datetime(impression.date)

        temp =  impression.set_index("index") 
        if n==0:
            print(n)

            data = temp['2017-06-13':'2017-06-18']
        if n==1:
            data = temp['2017-06-19']
        return data

In [752]:
train_data_total = selecttraintest(grouped_rich_impression,0)

0


/Users/XiHUANG/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [753]:
test_data_total = selecttraintest(grouped_rich_impression,1)

/Users/XiHUANG/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


### Re-arrange articles from just one day(2017-06-19), obtain related context vector, this data set as the test data set

In [754]:
def contextvectorfun(grouped_rich_impression, n ):
    
    
    traincandidate = pd.DataFrame()
    shrinkdate_impression = grouped_rich_impression
    shrinkdate_impression = shrinkdate_impression.reset_index()
    flat_date_impression = pd.DataFrame(
    data=[(index, tags)
         for index, row in shrinkdate_impression.iterrows()
         for tags in row.tags],
    columns=['index', 'tags']).set_index("index")
    
    contextvector = feature_matrix(flat_date_impression, 'tags')
    
    try1 = []
    for item in contextvector.toarray():
        try1.append(item)
        
    shrinkdate_impression["contextvector"] = try1
    
    #del shrinkdate_impression["index"]
    
    shrinkdate_impression["index"] = pd.to_datetime(shrinkdate_impression.date)

    temp =  shrinkdate_impression.set_index("index") 
    if n==0:
        temp = temp['2017-06-13':'2017-06-18']
        
        traincandidate = temp
        
        data = temp.drop_duplicates(subset=['contentId'], keep = 'first').reset_index()

        del data["index"]

        
    if n==1:
        
        data = temp['2017-06-19']
        
        #data = temp.drop_duplicates(subset=['contentId'], keep = 'first').reset_index()

        #del data["index"]
        
        
    
    return traincandidate,data



In [755]:
traincandidate,trainaction = contextvectorfun(grouped_rich_impression,0)


In [756]:
len(traincandidate[traincandidate["isClicked"]==1])/len(traincandidate)

0.02609583359918331

In [757]:
traincandidate.frontPageViewId.nunique()

2524

In [758]:
_,testcadidate = contextvectorfun(grouped_rich_impression,1)

### CCM

In [759]:
def sat(impressions):
    
    up = impressions.groupby(["contentId"]).aggregate({"satisfiedlist":sum})
    
    down = impressions.groupby(["contentId"]).aggregate({"isClicked":sum})
    
    up["sat"] = up.satisfiedlist.values/down.isClicked.values
    
    
    result = up.fillna(0)
    
    result = result.sat.to_dict()
    
    return result

### tua probability calculate

In [760]:
shrinkdataset = grouped_rich_impression[["date","frontPageViewId","contentPosition","contentId","satisfiedlist","isClicked"]]

In [761]:


def tua_inf(newtua_impression):
    
    #newtua_impression = grouped_rich_impression[["frontPageViewId","contentPosition","contentId","satisfiedlist","isClicked"]]
    
    #down = len(set(process_dataset.frontPageViewId.values))
    
    idx  = newtua_impression.groupby(['frontPageViewId'])["contentPosition"].transform(max) == newtua_impression["contentPosition"]

    newtua_impression = newtua_impression[idx]
    
    count = Counter(newtua_impression.contentPosition.values)
    
    a = list(count.values())
    
    n = len(a)
    
    examination_inf = {}
    
    for i in range(len(a)):
   
        examination_inf[i+1] = sum(a[i:n])
    
    temp = list( examination_inf.values())
    
    tua_probability = {}
    for i in range(1,62):
         tua_probability[i] = 0.02
    
    
    
    tua_probability[1] = 1
    
    for i in range(len(a)):
        if i == 0:
            tua_probability[i+1] = 1
        else:
            tua_probability[i+1] = temp[i]/temp[i-1]


    return  tua_probability
    

    

In [762]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 0) & (shrinkdataset['isClicked'] == 0))]

tua_s0c0 = tua_inf(dataset) # = tua 1

In [763]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 0) & (shrinkdataset['isClicked'] == 1))]

tua_s0c1 = tua_inf(dataset)

In [764]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 1) & (shrinkdataset['isClicked'] == 1))]

tua_s1c1 = tua_inf(dataset)

In [765]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 1) & (shrinkdataset['isClicked'] == 0))]

tua_s1c0 = tua_inf(dataset)

### satisficaiton probability of the train action

In [766]:
sat_prob_train = sat(train_data_total)

In [767]:
sat_prob_test = sat(test_data_total)

### CTR of the train action

In [768]:
def CTRperaction(grouped_rich_impression):

    up = grouped_rich_impression.groupby(["contentId"]).aggregate({"isClicked":sum})
    
    down = grouped_rich_impression.groupby(["contentId"]).aggregate({"isClicked":len})
    
    CTR = up.isClicked.values/down.isClicked.values
    
    up["CTR"] = CTR
    
    result = up.sort(columns="CTR")
    
    result = result.CTR.to_dict()
    
    return result

In [769]:
CTR_TRAIN = CTRperaction(train_data_total)

/Users/XiHUANG/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


### Bandit by myself

In [976]:
import random as rand


#sat_state = []
#click_state = []

#position = 1

def rewardactionfun(Q,sat_prob,examination,epsilon):
    
    
    if rand.random() > epsilon:
        d = dict((k, v) for k, v in Q.items() if v == max(Q.values()))
       
        
        action = rand.choice(list(d.keys()))
    else:
        action = rand.choice(list(Q.keys()))

    reward = sat_prob[action]*examination
    return action,reward

In [977]:
from sklearn.metrics.pairwise import cosine_similarity
def selectcadidate(trainvalue, candidate):
    S = defaultdict(float)
    for item in list(candidate.keys()):
    
       S[item]= float(1 - spatial.distance.cosine(list(trainvalue), list(candidate[item])) )
    
    if len(S) == 0:
        
        result = random.choice(list(candidate.keys()))
    else:
    
        result = max(S.items(), key=operator.itemgetter(1))[0]
    
   
        
    
    return result
        

In [978]:
def stateidentify(satvalue,reward):
   
    if(satvalue >= parameter_threshold):
        #sat_state.append(1)
        sat_state = 1
    else:

        #sat_state.append(0)
        sat_state = 0

    if(reward >= parameter_threshold):
        #click_state.append(1)
        click_state = 1
    else:
        #click_state(0)
        click_state = 0
    return click_state,sat_state



In [979]:
def actionvalueupdate(alpha,action,reward,Q,count):
    copy_Q = Q
    old_Q = copy_Q[action]
    Q[action] = (old_Q + (alpha*(reward-old_Q)))/count
    Q = {k: Q[k] for k in Q if not isnan(Q[k])}
    
    return Q

In [980]:

def examinationfunc(position,pre_sat_state,pre_click_state):
    
    if pre_sat_state == 1 and pre_click_state == 1:
        return tua_s1c1[position]
    if pre_sat_state == 0 and pre_click_state == 1:
        return tua_s0c1[position]
    if pre_sat_state == 0 and pre_click_state == 0:
        return tua_s0c0[position]
    if pre_sat_state == 1 and pre_click_state == 0:
        return tua_s1c0[position]

### Implement the bandit

In [1004]:
Q = CTR_TRAIN

In [1005]:
def candidateslection(impression):
    result = impression.set_index("contentId").contextvector.to_dict()
    return result

In [1006]:
def cadidateisclick(impression):
    result = impression.set_index("contentId").isClicked.to_dict()
    return result
    

In [1007]:
traincontext = trainaction.set_index("contentId").contextvector.to_dict()

In [1008]:
len(list(traincontext.values())[0])

904

In [1009]:
def inputvalue(df):
    tmp = df.groupby("frontPageViewId").apply(lambda x:candidateslection(x))

    tmp = tmp.to_frame(name="article")

    global_candidate = tmp["article"].to_dict()
    return global_candidate

In [1010]:
global_candidate = inputvalue(testcadidate)

In [1011]:
global_train = inputvalue(traincandidate)

In [1012]:
len(global_train)

2524

In [1013]:
tmp = testcadidate.groupby("frontPageViewId").apply(lambda x:cadidateisclick(x))

tmp = tmp.to_frame(name="click")

test_click =  tmp["click"].to_dict()
 

In [1014]:
global_candidate = inputvalue(testcadidate)
test_candidate = global_candidate

In [1015]:
train_candidate = global_train

In [1016]:
parameter_threshold = 0.6
parameter_alpha = 0.15
epsilon = 0
#countdic = defaultdict(int)
new_frontpage = pd.DataFrame()

### train model

In [1017]:
def CCM_BANDIT(sessioninput,Q,flag):
    total_predictdataset = pd.DataFrame()
    for item in list(sessioninput.keys()):
        countdic = defaultdict(int)
        position=1
        #print(position)
        candidate = sessioninput[item]
        while(len(candidate) != 0):

            if position == 1:
                Q_pre = Q
                #print(position)
                #Q_0 = CTR_TRAIN
                action,reward = rewardactionfun(Q_pre,sat_prob_train,1,epsilon)
                countdic[action]+=1
                choosefromcadidate = selectcadidate(traincontext[action], candidate)
                click_state,sat_state = stateidentify(sat_prob_train[action],reward)
                #print(test_click[item][choosefromcadidate])
                if flag == 0:
                    new_frontpage= pd.DataFrame({"position":[position],"frontpageid":item ,"contentid":choosefromcadidate,
                                             "probablectr":[reward],
                                            })
                if flag == 1:
                    new_frontpage= pd.DataFrame({"position":[position],"frontpageid":item ,"contentid":choosefromcadidate,
                                             "probablectr":[reward],
                                            "isclickreal": [test_click[item][choosefromcadidate]]})
                Q = actionvalueupdate(parameter_alpha,action,reward,Q_pre,countdic[action])
                del candidate[choosefromcadidate]
                position+=1
            else:


                Q_pre = Q

                examination = examinationfunc(position,sat_state,click_state)

                if examination == 0:
                    break
                else:

                    action,reward = rewardactionfun(Q_pre,sat_prob_train,examination,epsilon)
                    choosefromcadidate = selectcadidate(traincontext[action], candidate)
                    #print(action)
                    countdic[action] += 1
                    click_state,sat_state = stateidentify(sat_prob_train[action],reward)
                    if flag == 0:
                        new_frontpage = new_frontpage.append(pd.DataFrame({"position":[position],"frontpageid":item, "contentid":choosefromcadidate,
                                                                       "probablectr":[reward],
                                                                       
                                    }))
                    if flag == 1:
                        new_frontpage = new_frontpage.append(pd.DataFrame({"position":[position],"frontpageid":item, "contentid":choosefromcadidate,
                                                                       "probablectr":[reward],
                                                                       "isclickreal": [test_click[item][choosefromcadidate]]
                                    }))
                    Q = actionvalueupdate(parameter_alpha,action,reward,Q_pre,countdic[action])
                    del candidate[choosefromcadidate]
                    position+=1
        total_predictdataset = total_predictdataset.append(new_frontpage)
    return total_predictdataset, Q

In [1018]:
_,Q = CCM_BANDIT(train_candidate,Q,0)

In [1019]:
total_predictdataset,_Q = CCM_BANDIT(test_candidate,Q,1)

### evaluation -- map

In [1020]:
def truemap(df):
    maplist = []
    maxposition = max(list(df.contentPosition.values))
    #print(maxposition)
    clicklist = [0]*maxposition
    for item in df[df["isClicked"] == 1].contentPosition.values:
        clicklist[item-1] = 1
    for i in range(maxposition):
        #print(i)
        n = i+1
        maplist.append(sum(clicklist[:n])/n)
    
    result = sum(maplist)/len(maplist)
    return result
        

In [1021]:
def predictmap(df):
    maplist = df.isclickreal.cumsum().values / df.position.values
    result = maplist.mean()
    return result
    
    

In [1022]:
ap_true = testcadidate.groupby("frontPageViewId").apply(lambda x:truemap(x))

ap_true = ap_true.to_frame(name = "ap").reset_index()

ap_true = ap_true[ap_true["ap"] != 0.0]

ap_true["represent_frontpage"] = list(ap_true.index.values)

ap_predict = total_predictdataset.groupby("frontpageid").apply(lambda x:predictmap(x))

ap_predict = ap_predict.to_frame(name="ap").reset_index()

ap_predict = ap_predict[ap_predict["ap"] != 0.0]

ap_predict["represent_frontpage"] = list(ap_predict.index.values)

In [1023]:
map_true = ap_true.ap.values.sum()/len(ap_true)

map_predict = ap_predict.ap.values.sum()/len(ap_predict)

improvement = (map_predict - map_true)/map_true

print("map true %s", map_true)

print("map predict %s", map_predict)

print("improvement %s", improvement)

map true %s 0.119271991151
map predict %s 0.165651911444
improvement %s 0.388858438981


### evaluation--myself

In [1024]:
newfrontpage = total_predictdataset.groupby(["position"])["contentid"].last()

new_frontpage = newfrontpage.to_frame().reset_index()

In [1025]:
def generalexam(grouped_rich_impression):
    #n = len(a)
    idx  = grouped_rich_impression.groupby(['frontPageViewId'])["contentPosition"].transform(max) == grouped_rich_impression["contentPosition"]

    temp = grouped_rich_impression[idx]

    result = {}

    count = Counter(temp.contentPosition.values)

    list1 = [i+1 for i in range(61)]
    #nn = len(a)

    for item in list1:
        if item not in list(count.keys()):
             count[item] = 0

    a = list(count.values())  

    up = len(set(grouped_rich_impression.frontPageViewId.values))

    for i in range(len(list1)):

        result[i+1] = (sum(a[i:len(list1)])/up+0.001)
        
    return result
   


In [1026]:
general_exam = generalexam(grouped_rich_impression)

In [1027]:
def evaluationmy(testcadidate,new_frontpage,general_exam):
    oldfrontpage = testcadidate.groupby(["contentPosition"])["contentId"].last()

    oldfrontpage = oldfrontpage.to_frame().reset_index()
    
    
    position_content = oldfrontpage.set_index("contentId").contentPosition.to_dict()
    
    position_content_predict = new_frontpage.set_index("contentid").position.to_dict()
    
    true_crt = []
    for item in position_content.keys():

        true_crt.append(sat_prob_test[item]*general_exam[position_content[item]])
        
    pretect_rocrt = []
    ave_true_crt = sum(true_crt)/len(true_crt)
    
    for item in position_content_predict.keys():

        pretect_rocrt.append(sat_prob_test[item]*general_exam[position_content_predict[item]])
    
    ave_pre_crt = sum(pretect_rocrt)/len(pretect_rocrt)

    improvement1 = (ave_pre_crt-ave_true_crt)/ave_true_crt

   
    
    
    
    return improvement1


    

In [1001]:
len(testcadidate)

7035

In [1028]:
evaluationmy(testcadidate,new_frontpage,general_exam)

1.0370371375817649

### epsilon greedy

In [802]:
def rewardactionfungreedy(Q,sat_prob,epsilon):
    
    
    if rand.random() > epsilon:
        d = dict((k, v) for k, v in Q.items() if v == max(Q.values()))
       
        
        action = rand.choice(list(d.keys()))
    else:
        action = rand.choice(list(Q.keys()))

    reward = sat_prob[action]
    return action,reward

In [884]:
def actionvalueepsilonupdate(alpha,action,reward,Q):
    copy_Q = Q
    old_Q = copy_Q[action]
    Q[action] = old_Q + (alpha*(reward-old_Q))
    Q = {k: Q[k] for k in Q if not isnan(Q[k])}
    
    return Q

In [885]:
trainaction["initial"] = 0.0

In [934]:
content_Q = trainaction.set_index('contentId')['initial'].to_dict()

content_vector = trainaction.set_index('contentId')['contextvector'].to_dict()

In [936]:
Q = content_Q

In [937]:

time = 1
    
while(time<=10000):

        
            Q_pre = Q
            #print(position)
            #Q_0 = CTR_TRAIN
            action,reward = rewardactionfungreedy(Q_pre,sat_prob_train,epsilon=0.01)
            Q = actionvalueepsilonupdate(parameter_alpha,action,reward,Q_pre)
            
        
            time+=1
   


In [938]:
Q = sorted(Q.items(), key=lambda kv: kv[1], reverse=True)

In [940]:
from random import uniform, random, choice, sample
def selectcadidate_e(trainvalue, candidate):
            
        
    
    
                S = defaultdict(float)
                resultdic = defaultdict(float)
                for item in list(candidate.keys()):
                            #print(item)


                        S[item]= float(1 - spatial.distance.cosine(list(trainvalue), list(candidate[item]))) 



                if len(S) == 0:
                    result = rand.choice(list(candidate.keys()))

                else:
                    result = max(S.items(), key=operator.itemgetter(1))[0]
                
                return result

In [968]:
test_data = testcadidate.drop_duplicates(subset=['contentId'], keep = 'first').reset_index()

test_vector = test_data.set_index('contentId')['contextvector'].to_dict()

In [969]:
def generalbandit(inputvalue,item):
    result_frontpage = pd.DataFrame()

    candidate = inputvalue

    for i in range(len(Q)):
            if len(candidate) == 0:
                break
            else:

                trainitem = Q[i][0]
                article = selectcadidate_e(content_vector[trainitem], candidate)
                        #print(article)
                result_frontpage = result_frontpage.append(pd.DataFrame({"frontpageid":item,"contentid": article,"value":Q[i][1],"position":[i+1],"isclickreal": [test_click[item][article]]}))

                del candidate[article] 
        #total_predictdataset = total_predictdataset.append(result_frontpage)
    return result_frontpage

In [970]:
global_candidate = inputvalue(testcadidate)
test_candidate = global_candidate

In [971]:
totalfront = pd.DataFrame()
for item in list(test_candidate.keys()):
        
        #print(position)
        candidate = test_candidate[item]
        totalfront=totalfront.append(generalbandit(candidate,item))

In [812]:
result_frontpage = result_frontpage[:61]

In [972]:
def evaluationlb(testcadidate,new_frontpage,general_exam):
    contentlist = list(testcadidate.contentId.values)
    
    idx = new_frontpage.contentid.isin(contentlist)
    
    new_frontpage = new_frontpage[idx]
    
    
    oldfrontpage = testcadidate.groupby(["contentPosition"])["contentId"].last()

    oldfrontpage = oldfrontpage.to_frame().reset_index()
    
    position_content = oldfrontpage.set_index("contentId").contentPosition.to_dict()
    
    position_content_predict = new_frontpage.set_index("contentid").position.to_dict()
    
    true_crt = []
    for item in position_content.keys():

        true_crt.append(sat_prob_test[item]*general_exam[position_content[item]])
    ave_true_crt = sum(true_crt)/len(true_crt)
        
    pretect_rocrt = []
    
    for item in position_content_predict.keys():

        pretect_rocrt.append(sat_prob_test[item]*general_exam[position_content_predict[item]])
    
    ave_pre_crt = sum(pretect_rocrt)/len(pretect_rocrt)

    improvement1 = (ave_pre_crt-ave_true_crt)/ave_true_crt

   
    
    #improvement2 = (ave_predict_ctr-ave_true_crt)/ave_true_crt
    
    return improvement1


    

In [999]:
evaluationmy(testcadidate,totalfront,general_exam)

2.467700472094867

In [973]:
evaluationlb(testcadidate,totalfront,general_exam)

2.467700472094867

### map epsilon

In [967]:


ap_predict =totalfront.groupby("frontpageid").apply(lambda x:predictmap(x))

ap_predict = ap_predict.to_frame(name="ap").reset_index()

ap_predict = ap_predict[ap_predict["ap"] != 0.0]

ap_predict["represent_frontpage"] = list(ap_predict.index.values)

map_predict = ap_predict.ap.values.sum()/len(ap_predict)

improvement = (map_predict - map_true)/map_true

print("map true %s", map_true)

print("map predict %s", map_predict)

print("improvement %s", improvement)

map true %s 0.119271991151
map predict %s 0.173521058299
improvement %s 0.454834924987
